# Grouping, summary functions and sorting

### Initialization

In [2]:
import pandas as pd

df = pd.read_csv('data/titanic.csv')
wines = pd.read_csv('data/wines.csv', index_col=0)

df.head(3)
wines.head(3)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


### Groupby

In [3]:
# Groupby requieres a summary function like count() to show results (otherwise it just throws a DataFrameGroupby object)
df.groupby('Pclass').count()

# This will count the number of PassengerId's in every Pclass (Pclass becomes the index)
df.groupby('Pclass')['PassengerId'].count()

Pclass
1    216
2    184
3    491
Name: PassengerId, dtype: int64

### Groupby summary functions

In [7]:
# Count/Size
df.groupby('Pclass').count()         # Returns a df. It counts every value (in every column) present in the grouping (performed over a DataFrameGroupBy obj)
df.groupby('Pclass').size()          # Count() isn't always best, for it'll return a whole DataFrame with the counted values for each column, whereas size() will count the total number of entries in given group.
df.groupby('Pclass').Pclass.count()  # This is the count() equivalent to df.groupby('Pclass').size()

# Max/Min
df.groupby('Pclass').Age.min()
df.groupby('Pclass').Age.max()

# Sum
df.groupby('Pclass').Age.sum()

# Describe
df.groupby('Pclass').Age.describe()

# Mean/median
df.groupby('Pclass').Age.mean()
df.groupby('Pclass').median()

# Std/Var
df.groupby('Pclass').var()
df.groupby('Pclass').std()

,PassengerId,Survived,Age,SibSp,Parch,Fare
Pclass,,,,,,
1,246.737616,0.484026,14.802856,0.611898,0.693997,78.380373
2,250.852161,0.500623,14.001077,0.601633,0.690963,13.417399
3,264.441453,0.428949,12.495398,1.374883,0.888861,11.778142


### Groupby with apply & lambda

In [9]:
# Apply function returns what you will see after all the grouping is done
def test_func(df):
    return df.price.mean()

# Iterations of apply are mini DataFrames, like slices of the original, each slice is the result of grouping and subgrouping the original df.
wines.groupby(['country', 'province']).apply(test_func)

country    province        
Argentina  Mendoza Province    25.053317
           Other               21.203774
Armenia    Armenia             14.500000
Australia  Australia Other     12.427966
           New South Wales     25.623529
                                 ...    
Uruguay    Juanico             48.583333
           Montevideo          26.090909
           Progreso            24.272727
           San Jose            30.000000
           Uruguay             22.083333
Length: 425, dtype: float64

In [10]:
# You can also use summary functions after the apply
wines.groupby(['country', 'province']).apply(test_func).sum()

11532.233156621889

### Grouby apply and summary functions
The difference between DataFrameGroupBy/SeriesGroupBy and regular DataFrame/Series objects is that when performing operations (like apply) or using summary functions on a GroupBy object, you'll perform the computation on the every single one of the groups created (sub-dataframes).<br>

When performing operations on regular DataFrame/Series objects you are performing them on each of the values of the whole df (not sub-dataframes).

In [11]:
# Right after the groupby you'll get a DataFrameGroupby or SeriesGroupby object
wines.groupby('country').__class__  # Throws DataFrameGroupBy
wines.groupby('country').province.__class__  # Throws a SeriesGroupBy

# Right after the apply, you'll get either a regular DataFrame of a Series object. 
wines.groupby('country').apply(lambda sub_df: sub_df.loc[sub_df.points.idxmax()])  # Throws regular DataFrame
wines.groupby('country').apply(lambda sub_df: sub_df.loc[sub_df.points.idxmax()]).price.head()  # Throws regular Series

country
Argentina                 120.0
Armenia                    15.0
Australia                 350.0
Austria                     NaN
Bosnia and Herzegovina     12.0
Name: price, dtype: float64

### Groupby with agg functions

In [12]:
# Agg takes an array of keyword summary functions (returns a DataFrame)
wines.groupby('country').price.agg(['min', 'max'])

# It can also take a dictionary where key is the column and value is the array of callbacks to perform over 
wines.groupby('country').agg({'price': ['mean', 'std'], 'points': ['min', 'max']})  

price            points     
                             mean        std    min  max
country                                                 
Argentina               24.510117  23.430122     80   97
Armenia                 14.500000   0.707107     87   88
Australia               35.437663  49.049458     80  100
Austria                 30.762772  27.224797     82   98
Bosnia and Herzegovina  12.500000   0.707107     85   88
Brazil                  23.765957  11.053649     80   89
Bulgaria                14.645390   9.508744     80   91
Canada                  35.712598  19.658148     82   94
Chile                   20.786458  21.929371     80   95
China                   18.000000        NaN     89   89
Croatia                 25.450704  12.993173     81   91
Cyprus                  16.272727   2.831639     85   89
Czech Republic          24.250000  12.053856     84   89
Egypt                         NaN        NaN     84   84
England                 51.681159  14.858641     89   95
France                  41.139120  73.769404     80  100
Georgia                 19.321429   7.642873     82   92
Germany                 42.257547  62.860308     81   98
Greece                  22.364425  10.610780     82   93
Hungary                 40.648276  69.451075     81   97
India                   13.333333   3.640055     87   93
Israel                  31.768916  18.898287     80   94
Italy                   39.663770  38.052479     80  100
Lebanon                 30.685714  18.038800     82   91
Luxembourg              23.333333   4.633213     88   90
Macedonia               15.583333   1.443376     84   89
Mexico                  26.785714  17.755091     80   92
Moldova                 16.745763   9.522375     82   91
Morocco                 19.500000   5.903546     85   93
New Zealand             26.931785  17.101083     80   95
Peru                    18.062500  13.664888     80   86
Portugal                26.218256  41.175241     80  100
Romania                 15.241667  30.286863     82   92
Serbia                  24.500000  10.264679     86   89
Slovakia                16.000000        NaN     87   87
Slovenia                24.812500  13.982895     82   92
South Africa            24.668987  21.843064     82   95
Spain                   28.215275  34.662397     80   98
Switzerland             85.285714  70.063577     83   90
Turkey                  24.633333  13.799894     84   92
US                      36.573464  27.088857     80  100
Ukraine                  9.214286   2.190138     82   88
Uruguay                 26.403670  18.483786     80   92

### Reseting indexes

In [13]:
# You can reset indexes take grouping indexes and spread them as values across all grouped entries (e.g.: for saving in excel later)
wines.groupby(['country', 'province']).mean().reset_index()

,country,province,points,price
0,Argentina,Mendoza Province,86.826593,25.053317
1,Argentina,Other,86.001866,21.203774
2,Armenia,Armenia,87.500000,14.500000
3,Australia,Australia Other,85.518367,12.427966
4,Australia,New South Wales,87.694118,25.623529
...,...,...,...,...
420,Uruguay,Juanico,86.333333,48.583333
421,Uruguay,Montevideo,88.272727,26.090909
422,Uruguay,Progreso,86.818182,24.272727
423,Uruguay,San Jose,84.000000,30.000000


### Unstacking and stacking
This is another method for spreading out groups. This one is particulary usefull if one needs the take into account the categories even if there is no values in a given group belonging to it. <br><br>
[Reference link](https://stackoverflow.com/questions/61243138/pandas-groupby-count-and-fill-none-count-as-0#answer-61243200)

In [14]:
# Gives the number of entries in each of the groups
wines.groupby(['country', 'province']).size()  

# It'll spread the groups and fill the missing values with 0's
wines.groupby(['country', 'province']).size().unstack(fill_value=0)

# It'll spread the groups, fill the missing values with 0's and stack them once more
wines.groupby(['country', 'province']).size().unstack(fill_value=0).stack()

country    province                 
Argentina  Achaia                       0
           Aconcagua Costa              0
           Aconcagua Valley             0
           Aegean                       0
           Agioritikos                  0
                                       ..
Uruguay    Zenata                       0
           Österreichischer Perlwein    0
           Österreichischer Sekt        0
           Štajerska                    0
           Župa                         0
Length: 18275, dtype: int64